# 数据预处理

In [1]:
import arcpy
from arcpy.sa import *

In [2]:
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\UFZs.gdb"

## 土地利用

### 研究区域裁剪

In [13]:
# Define the paths to your datasets
ring = r"E:\stu1\Projects\POI2023\UFZs.gdb\ring3"
# poi_data = r"E:\stu1\Projects\POI2023\POI2023.gdb\MergedPOI23"
# road_network_data = r"E:\stu1\Projects\POI2023\POI2023.gdb\UFZs23\road23"
land_use_data = r"E:\stu1\UFZs\lucc\2020.tif"

In [14]:
# Define the output paths
#clipped_poi = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_poi"
#clipped_road_network = r"E:\stu1\Projects\POI2023\POI2023.gdb\clipped_road"
clipped_land_use = r"E:\stu1\Projects\POI2023\UFZs.gdb\lucc20"

In [ ]:
ring = r"E:\stu1\Projects\POI2023\UFZs.gdb\ring3"

In [13]:
land_use_data = r"E:\stu1\UFZs\lucc\2018.tif"
clipped_land_use = r"E:\stu1\Projects\POI2023\UFZs.gdb\lucc18"

In [14]:
# Perform the extract by mask operation
masked_raster = ExtractByMask(land_use_data,ring)
masked_raster.save(clipped_land_use)

### 土地利用数据投影

## 路网

In [19]:
# 裁剪
road_network_data = r"E:\stu1\UFZs\13-23OSM\13\gis_osm_roads_free_1.shp"
clipped_road = r"E:\stu1\Projects\POI2023\UFZs.gdb\road13"

# Clip the datasets
arcpy.Clip_analysis(road_network_data, ring, clipped_road)

<Result 'E:\\stu1\\Projects\\POI2023\\UFZs.gdb\\road13'>

In [38]:
# 裁剪
road_network_data = r"E:\stu1\UFZs\13-23OSM\23\gis_osm_roads_free_1.shp"
clipped_road = r"E:\stu1\Projects\POI2023\UFZs.gdb\road23"

In [39]:
# Clip the datasets
arcpy.Clip_analysis(road_network_data, ring, clipped_road)

<Result 'E:\\stu1\\Projects\\POI2023\\UFZs.gdb\\road23'>

## POI

In [3]:
# 定义不同功能区对应的POI类型
typecode_ranges = {
    'R': [str(num) for num in range(120300, 120305)],   # 居住用地
    'A': [f"{num:06d}" for num in range(80000, 80007)] +  # 从 080000 到 080006
        [f"{num:06d}" for num in range(80100, 80120)] +  # 从 080100 到 080119
        [f"{num:06d}" for num in range(80200, 80203)] +  # 从 080200 到 080202
        [f"{num:06d}" for num in range(80300, 80309)] +  # 从 080300 到 080308
        [f"{num:06d}" for num in range(141100, 141206)],  # 从 141100 到 141205  # 公共管理与公共服务设施用地
    'B': [f"{num:06d}" for num in range(50100, 50124)] +  # 从 050100 到 050123
        [f"{num:06d}" for num in range(50200, 50218)] +  # 从 050200 到 050217
        [f"{num:06d}" for num in range(60100, 60104)] +  # 从 060100 到 060103
        [f"{num:06d}" for num in range(60400, 60410)] +  # 从 060400 到 060409
        [f"{num:06d}" for num in range(61000, 61002)] +  # 从 061000 到 061001
        [f"{num:06d}" for num in range(160400, 160409)] +  # 从 160400 到 160408
        [f"{num:06d}" for num in range(160500, 160502)],   # 从 160500 到 160501   # 商业服务业设施用地
    'M': ["120100"] +  # 单独的值
        ["120202"] +  # 单独的值
        ["170000"] +  # 单独的值
        [f"{num:06d}" for num in range(170100, 170101)] +  # 从 170100 到 170100
        [f"{num:06d}" for num in range(170200, 170209)] +  # 从 170200 到 170208
        [f"{num:06d}" for num in range(170300, 170301)] +  # 从 170300 到 170300
        [f"{num:06d}" for num in range(170400, 170409)],   # 从 170400 到 170408    # 工业用地
    'S': [f"{num:06d}" for num in range(150104, 150108)] +  # 从 150104 到 150107
        [f"{num:06d}" for num in range(150200, 150211)] +  # 从 150200 到 150210
        [f"{num:06d}" for num in range(150300, 150308)] +  # 从 150300 到 150307
        ["180000"] +                                    # 单独的值
        [f"{num:06d}" for num in range(180100, 180105)] +  # 从 180100 到 180104
        [f"{num:06d}" for num in range(180200, 180204)] +  # 从 180200 到 180203
        [f"{num:06d}" for num in range(180300, 180303)] +  # 从 180300 到 180302
        ["180400", "180500"],                              # 单独的值 # 交通与道路设施用地
    'G': ["110000"] +  # 单独的值
        [f"{num:06d}" for num in range(110100, 110107)] +  # 从 110100 到 110106
        [f"{num:06d}" for num in range(110200, 110211)]    # 从 110200 到 110210 # 绿地与广场用地
}


In [6]:
for year in range(12, 24):
    # 输入的POI要素类
    input_feature_class = f"E:\stu1\Projects\POI2023\POIs.gdb\POI{year}"
    
    # 创建一个新的要素图层
    input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)
    
    for category, codes in typecode_ranges.items():
        # 创建查询
        query = "typecode IN ('{}')".format("', '".join(codes))

        # 按类型代码筛选POI
        arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

        # 保存筛选结果为新的要素类
        output_feature_class = f"POI_{year}{category}"
        arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

        print(f"{year}年分类保存完成：", output_feature_class)

        # 清除当前的选择
        arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

    print(f"{year}年所有分类完成")


12年分类保存完成： POI_12R
12年分类保存完成： POI_12A
12年分类保存完成： POI_12B
12年分类保存完成： POI_12M
12年分类保存完成： POI_12S
12年分类保存完成： POI_12G
12年所有分类完成


ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


In [ ]:
for year in range(12, 18):
    # 输入的POI要素类
    input_feature_class = f"E:\\stu1\\Projects\\POI2023\\POIs.gdb\\POI{year}"

    # 创建一个新的要素图层
    input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)

    for category, codes in typecode_ranges.items():
        # 根据年份和字段类型构建查询
        if year in [13, 15, 16, 17]:
            # typecode为double类型
            query = "typecode IN ({})".format(", ".join(map(str, codes)))
        else:
            # typecode为字符串类型
            query = "typecode IN ('{}')".format("', '".join(codes))

        # 对于14年、15年、16年使用大写字段名
        if year in [14, 15, 16]:
            query = query.replace("typecode", "TYPECODE")

        print(f"查询语句: {query}")  # 打印查询语句检查

        # 按类型代码筛选POI
        arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

        # 保存筛选结果为新的要素类
        output_feature_class = f"POI_{year}{category}"
        arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

        print(f"{year}年分类保存完成：{output_feature_class}")

        # 清除当前的选择
        arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

    print(f"{year}年所有分类完成")


# 筛选

In [ ]:
# 投影

In [ ]:
for year in range(13, 24):
    

# 为街区添加功能识别指标字段

In [8]:
# block23.shp 的路径
block23_path = r"E:\stu1\Projects\POI2023\POI2023.gdb\block23\block23"

# 定义六个新字段
type_ratio_fields = ["R_Ratio", "A_Ratio", "B_Ratio", 
                     "M_Ratio", "S_Ratio", "G_Ratio"]

# 添加新字段
for field in type_ratio_fields:
    arcpy.AddField_management(block23_path, field, "DOUBLE")

print("字段添加完成")


字段添加完成


# 数据分析

In [9]:
# temp_stats_table 路径
temp_stats_table_path = r"E:\stu1\Projects\POI2023\POI2023.gdb\temp_stats"

In [10]:
# 准备一个字典来收集每个街区的类型比例值
block_type_ratios = {}

# 从 temp_stats_table 收集每个街区的类型比例值
with arcpy.da.SearchCursor(temp_stats_table_path, ["BlockID", "type", "Type_Ratio"]) as cursor:
    for row in cursor:
        block_id = row[0]
        type_code = row[1]
        type_ratio = row[2]

        if block_id not in block_type_ratios:
            block_type_ratios[block_id] = {field: 0 for field in type_ratio_fields}
        
        field_name = f"{type_code}_Ratio"
        block_type_ratios[block_id][field_name] = type_ratio
        


In [11]:
# 更新 block23 要素类中的字段值
with arcpy.da.UpdateCursor(block23_path, ["BlockID"] + type_ratio_fields) as cursor:
    for row in cursor:
        block_id = row[0]

        # 如果这个街区的类型比例值存在于字典中，更新它们
        if block_id in block_type_ratios:
            for i, field in enumerate(type_ratio_fields, 1):
                row[i] = block_type_ratios[block_id][field]
            cursor.updateRow(row)

print("block23 要素类中的类型比例字段更新完成")

block23 要素类中的类型比例字段更新完成


# 识别单一功能区

In [20]:
# 添加一个新字段来标记单一功能区
dominant_field = "Dominant_Type"
arcpy.AddField_management(block23_path, dominant_field, "TEXT")

<Result 'E:\\stu1\\Projects\\POI2023\\POI2023.gdb\\block23\\block23'>

In [21]:
# 标记 Dominant_Type 字段
with arcpy.da.UpdateCursor(block23_path, ["BlockID"] + type_ratio_fields + [dominant_field]) as cursor:
    for row in cursor:
        # 获取除 BlockID 和 Dominant_Type 外的 Type_Ratio 值
        type_ratios = row[1:-1]
        
        # 检查是否所有 Type_Ratio 值都是 None
        if all(value is None for value in type_ratios):
            row[-1] = None  # 全部为 NULL，则标记为 NULL
        else:
            # 找出最大的 Type_Ratio（忽略 None 值）
            valid_ratios = [value for value in type_ratios if value is not None]
            max_ratio = max(valid_ratios) if valid_ratios else 0  # 如果没有有效值，使用 0 作为最大值

            if max_ratio > 0.5:
                # 获取最大 Type_Ratio 值的索引
                dominant_index = type_ratios.index(max_ratio)
                # 标记为对应的单一功能类型
                row[-1] = type_ratio_fields[dominant_index].replace("_Ratio", "")
            else:
                row[-1] = "Mixed"  # 如果没有类型比例大于 0.5，则为混合功能区
        cursor.updateRow(row)

print("Dominant_Type 字段更新完成")


Dominant_Type 字段更新完成


 检查

In [22]:
# 检查 Type_Ratio 字段和计算 DOMINANT_TYPE 字段的逻辑
count = 0
with arcpy.da.UpdateCursor(block23_path, ["BlockID"] + type_ratio_fields + [dominant_field]) as cursor:
    for row in cursor:
        count += 1
        if count <= 200:  # 只检查前200条记录
            print(f"BlockID {row[0]}: Type Ratios - {row[1:-1]}, Dominant Type - {row[-1]}")
            # 检查每个 Type_Ratio 值
            if all(value is None for value in row[1:-1]):
                print("All values are None. Setting Dominant Type to None.")
            else:
                valid_ratios = [value for value in row[1:-1] if value is not None]
                max_ratio = max(valid_ratios) if valid_ratios else 0
                if max_ratio > 0.5:
                    dominant_index = row[1:-1].index(max_ratio)  # 获取 Type_Ratio 最大值的索引
                    dominant_type = type_ratio_fields[dominant_index].replace("_Ratio", "")
                    print(f"Found a dominant type: {dominant_type}")
                else:
                    print("No dominant type found. Setting to Mixed.")


BlockID 1: Type Ratios - [0.2844759095321871, 0.2589143555074278, 0.30073414087764544, 0.15587559408273968, 0.0, 0.0], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 2: Type Ratios - [0.29710516562801664, 0.25753220102524094, 0.3111903839184352, 0.13417224942830713, 0.0, 0.0], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 3: Type Ratios - [0.1951019775024251, 0.17263857827595722, 0.28276532205836435, 0.3494941221632534, 0.0, 0.0], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 4: Type Ratios - [0.3260789312950058, 0.10991820072793636, 0.38919417269755113, 0.06871982075748553, 0.0, 0.10608887452202098], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 5: Type Ratios - [0.17261810307422207, 0.17456391212674358, 0.4709251367139519, 0.18189284808508244, 0.0, 0.0], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 6: Type Ratios - [0.197537177193157, 0.19976388239236034, 

BlockID 103: Type Ratios - [0.1999133794426916, 0.0, 0.7158247423196366, 0.0842618782376718, 0.0, 0.0], Dominant Type - B
Found a dominant type: B
BlockID 104: Type Ratios - [0.1276016949117206, 0.258080127878772, 0.27559055859679266, 0.08963847667046024, 0.0, 0.2490891419422546], Dominant Type - Mixed
No dominant type found. Setting to Mixed.
BlockID 105: Type Ratios - [None, None, None, None, None, None], Dominant Type - None
All values are None. Setting Dominant Type to None.
BlockID 106: Type Ratios - [0.0, 0.03562811838673548, 0.0, 0.04454872611175617, 0.9198231555015083, 0.0], Dominant Type - S
Found a dominant type: S
BlockID 107: Type Ratios - [None, None, None, None, None, None], Dominant Type - None
All values are None. Setting Dominant Type to None.
BlockID 108: Type Ratios - [None, None, None, None, None, None], Dominant Type - None
All values are None. Setting Dominant Type to None.
BlockID 109: Type Ratios - [None, None, None, None, None, None], Dominant Type - None
All v

# 尝试为核密度方法调整KDB结果

In [2]:
import arcpy
import numpy as np

# 设置工作环境和矢量文件路径
arcpy.env.workspace = r'E:\stu1\Projects\POI2023\POI2023.gdb'  
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'  # 请替换为你的矢量文件名

# 原始密度值字段名称
density_fields = ['KDB_M', 'KDB_G', 'KDB_S', 'KDB_B', 'KDB_R', 'KDB_A']
# 新的标准化字段名称
new_fields = ['KDB_M_Norm', 'KDB_G_Norm', 'KDB_S_Norm', 'KDB_B_Norm', 'KDB_R_Norm', 'KDB_A_Norm']

# 为每个密度值添加新字段用于存储标准化后的值
for new_field in new_fields:
    arcpy.AddField_management(feature_class, new_field, "FLOAT")

# 读取POI核密度值，并进行标准化
with arcpy.da.UpdateCursor(feature_class, density_fields + new_fields) as cursor:
    # 存储每种类型POI的密度值
    densities = {field: [] for field in density_fields}
    # 先读取所有值
    for row in cursor:
        for idx, field in enumerate(density_fields):
            densities[field].append(row[idx])
    
    # 计算每个字段的最小值、最大值和范围用于标准化
    min_max_range = {field: (np.min(values), np.max(values), np.max(values) - np.min(values)) for field, values in densities.items()}
    
    # 重置cursor以更新行
    cursor.reset()
    
    # 更新新字段的标准化值
    for row in cursor:
        for idx, field in enumerate(density_fields):
            # 获取最小值和范围
            min_val, max_val, range_val = min_max_range[field]
            # 当范围为0时，所有值都相同，可以直接设置为0或1
            if range_val == 0:
                normalized_val = 0  # 或1，视需求而定
            else:
                # 进行最小-最大标准化
                normalized_val = (row[idx] - min_val) / range_val
            
            # 更新标准化值
            row[idx + len(density_fields)] = normalized_val
        cursor.updateRow(row)


In [8]:
# 如果用norm字段计算类型比例
norm_fields = ['KDB_M_Norm', 'KDB_G_Norm', 'KDB_S_Norm', 'KDB_B_Norm', 'KDB_R_Norm', 'KDB_A_Norm']
prop_fields = ['N_M_Prop', 'N_G_Prop', 'N_S_Prop', 'N_B_Prop', 'N_R_Prop', 'N_A_Prop']

# 为比例值添加新字段
for prop_field in prop_fields:
    arcpy.AddField_management(feature_class, prop_field, "DOUBLE")

# 计算并更新比例值
with arcpy.da.UpdateCursor(feature_class, norm_fields + prop_fields) as cursor:
    for row in cursor:
        # 计算所有标准化值的总和
        total_norm = sum(row[:len(norm_fields)])
        # 避免除以零
        if total_norm > 0:
            # 计算每个类型的比例值并更新行
            for i in range(len(norm_fields)):
                row[len(norm_fields) + i] = row[i] / total_norm
        else:
            # 如果总和为0，则将比例值设为平均值或其他适当的默认值
            for i in range(len(norm_fields), len(norm_fields) + len(prop_fields)):
                row[i] = 1.0 / len(norm_fields)  # 或其他适当的值
        cursor.updateRow(row)

In [9]:
import arcpy

def identify_urban_function_with_details(feature_class, method_prefix='FD', thresholds=(0.5, 0.4)):
    """
    Identifies the urban function of each block, dynamically names output fields based on the method prefix,
    adds two new fields to the feature class: one for the urban function category and another for the specific leading categories.
    """
    # Dynamically create field names based on method_prefix
    function_field = f"{method_prefix}_UrbanFuncCat"
    leading_field = f"{method_prefix}_LeadingCat"

    # Ensure the new fields exist, if not, add them
    existing_fields = [f.name for f in arcpy.ListFields(feature_class)]
    if function_field not in existing_fields:
        arcpy.AddField_management(feature_class, function_field, "TEXT", field_length=255)
    if leading_field not in existing_fields:
        arcpy.AddField_management(feature_class, leading_field, "TEXT", field_length=255)

    # Define a dictionary to map field suffixes to abbreviations
    category_abbreviations = {
        'A_Prop': '公服',  # Residential
        'B_Prop': '商业',  # Commercial
        'R_Prop': '居住',  # Recreational
        'G_Prop': '绿地',  # Green Space
        'S_Prop': '交通',  # Services
        'M_Prop': '工业'   # Manufacturing
    }

    # Generate field names for the POI proportions based on the method prefix
    poi_fields = [f'{method_prefix}_{suffix}' for suffix in category_abbreviations.keys()]
    field_to_abbr = {f'{method_prefix}_{key}': val for key, val in category_abbreviations.items()}

    # Update cursor to classify each block and identify leading categories
    with arcpy.da.UpdateCursor(feature_class, poi_fields + [function_field, leading_field]) as cursor:
        for row in cursor:
            proportions = row[:-2]  # Exclude the output fields values
            high_threshold, low_threshold = thresholds
            
            # Identify categories exceeding the high threshold
            high_categories = [field_to_abbr[poi_fields[i]] for i, prop in enumerate(proportions) if prop > high_threshold]
            if high_categories:
                row[-2] = 'Single Dominant'
                row[-1] = '/'.join(high_categories)
            else:
                # Identify categories exceeding the low threshold
                low_categories = [field_to_abbr[poi_fields[i]] for i, prop in enumerate(proportions) if prop > low_threshold]
                if len(low_categories) == 1:
                    row[-2] = 'Single Mixed'
                    row[-1] = low_categories[0]
                elif len(low_categories) >= 2:
                    row[-2] = 'Dual Mixed'
                    row[-1] = '/'.join(low_categories[:2])  # Adjust based on preference
                else:
                    row[-2] = 'Multi Mixed'
                    row[-1] = 'MULTI'

            cursor.updateRow(row)

In [10]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'N'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

# 对核密度调整

In [13]:
import numpy as np
import arcpy

# 假设feature_class是你的街区特征类路径
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'

# 原始密度值字段名称
density_fields = ['KD_M', 'KD_G', 'KD_S', 'KD_B', 'KD_R', 'KD_A']
# 新的标准化字段名称
new_fields = ['KD_M_Norm', 'KD_G_Norm', 'KD_S_Norm', 'KD_B_Norm', 'KD_R_Norm', 'KD_A_Norm']

# 为每个密度值添加新字段用于存储标准化后的值
for new_field in new_fields:
    arcpy.AddField_management(feature_class, new_field, "FLOAT")

# 存储每种类型POI的密度值
densities = {field: [] for field in density_fields}

# 使用SearchCursor读取所有值
with arcpy.da.SearchCursor(feature_class, density_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            # 跳过None值
            if row[idx] is not None:
                densities[field].append(row[idx])

# 清理None值并计算最小值、最大值和范围
min_max_range = {}
for field, values in densities.items():
    cleaned_values = [x if x is not None else 0 for x in values]  # 清理None值
    min_val = np.min(cleaned_values)
    max_val = np.max(cleaned_values)
    range_val = max_val - min_val
    min_max_range[field] = (min_val, max_val, range_val)

# 使用UpdateCursor更新新字段的标准化值
with arcpy.da.UpdateCursor(feature_class, density_fields + new_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            min_val, max_val, range_val = min_max_range[field]
            value = row[idx] if row[idx] is not None else 0  # 处理None值
            # 当范围为0时，所有值都相同，可以直接设置为0或1
            normalized_val = (value - min_val) / range_val if range_val else 0
            # 更新标准化值
            row[idx + len(density_fields)] = normalized_val
        cursor.updateRow(row)


In [14]:
# 如果用norm字段计算类型比例
norm_fields = ['KD_M_Norm', 'KD_G_Norm', 'KD_S_Norm', 'KD_B_Norm', 'KD_R_Norm', 'KD_A_Norm']
prop_fields = ['N0_M_Prop', 'N0_G_Prop', 'N0_S_Prop', 'N0_B_Prop', 'N0_R_Prop', 'N0_A_Prop']

# 为比例值添加新字段
for prop_field in prop_fields:
    arcpy.AddField_management(feature_class, prop_field, "DOUBLE")

# 计算并更新比例值
with arcpy.da.UpdateCursor(feature_class, norm_fields + prop_fields) as cursor:
    for row in cursor:
        # 计算所有标准化值的总和
        total_norm = sum(row[:len(norm_fields)])
        # 避免除以零
        if total_norm > 0:
            # 计算每个类型的比例值并更新行
            for i in range(len(norm_fields)):
                row[len(norm_fields) + i] = row[i] / total_norm
        else:
            # 如果总和为0，则将比例值设为平均值或其他适当的默认值
            for i in range(len(norm_fields), len(norm_fields) + len(prop_fields)):
                row[i] = 1.0 / len(norm_fields)  # 或其他适当的值
        cursor.updateRow(row)

In [15]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'N0'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

# 为去丛聚核密度

In [16]:
import numpy as np
import arcpy

# 假设feature_class是你的街区特征类路径
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'

# 原始密度值字段名称
density_fields = ['DE_KD_M', 'DE_KD_G', 'DE_KD_S', 'DE_KD_B', 'DE_KD_R', 'DE_KD_A']
# 新的标准化字段名称
new_fields = ['DK_M_Norm', 'DK_G_Norm', 'DK_S_Norm', 'DK_B_Norm', 'DK_R_Norm', 'DK_A_Norm']

# 为每个密度值添加新字段用于存储标准化后的值
for new_field in new_fields:
    arcpy.AddField_management(feature_class, new_field, "FLOAT")

# 存储每种类型POI的密度值
densities = {field: [] for field in density_fields}

# 使用SearchCursor读取所有值
with arcpy.da.SearchCursor(feature_class, density_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            # 跳过None值
            if row[idx] is not None:
                densities[field].append(row[idx])

# 清理None值并计算最小值、最大值和范围
min_max_range = {}
for field, values in densities.items():
    cleaned_values = [x if x is not None else 0 for x in values]  # 清理None值
    min_val = np.min(cleaned_values)
    max_val = np.max(cleaned_values)
    range_val = max_val - min_val
    min_max_range[field] = (min_val, max_val, range_val)

# 使用UpdateCursor更新新字段的标准化值
with arcpy.da.UpdateCursor(feature_class, density_fields + new_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            min_val, max_val, range_val = min_max_range[field]
            value = row[idx] if row[idx] is not None else 0  # 处理None值
            # 当范围为0时，所有值都相同，可以直接设置为0或1
            normalized_val = (value - min_val) / range_val if range_val else 0
            # 更新标准化值
            row[idx + len(density_fields)] = normalized_val
        cursor.updateRow(row)


In [17]:
# 如果用norm字段计算类型比例
norm_fields = ['DK_M_Norm', 'DK_G_Norm', 'DK_S_Norm', 'DK_B_Norm', 'DK_R_Norm', 'DK_A_Norm']
prop_fields = ['N1_M_Prop', 'N1_G_Prop', 'N1_S_Prop', 'N1_B_Prop', 'N1_R_Prop', 'N1_A_Prop']

# 为比例值添加新字段
for prop_field in prop_fields:
    arcpy.AddField_management(feature_class, prop_field, "DOUBLE")

# 计算并更新比例值
with arcpy.da.UpdateCursor(feature_class, norm_fields + prop_fields) as cursor:
    for row in cursor:
        # 计算所有标准化值的总和
        total_norm = sum(row[:len(norm_fields)])
        # 避免除以零
        if total_norm > 0:
            # 计算每个类型的比例值并更新行
            for i in range(len(norm_fields)):
                row[len(norm_fields) + i] = row[i] / total_norm
        else:
            # 如果总和为0，则将比例值设为平均值或其他适当的默认值
            for i in range(len(norm_fields), len(norm_fields) + len(prop_fields)):
                row[i] = 1.0 / len(norm_fields)  # 或其他适当的值
        cursor.updateRow(row)

In [18]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'N1'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

# 为去从聚KDB标准化

In [19]:
import numpy as np
import arcpy

# 假设feature_class是你的街区特征类路径
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'

# 原始密度值字段名称
density_fields = ['DE_KDB_M', 'DE_KDB_G', 'DE_KDB_S', 'DE_KDB_B', 'DE_KDB_R', 'DE_KDB_A']
# 新的标准化字段名称
new_fields = ['DB_M_Norm', 'DB_G_Norm', 'DB_S_Norm', 'DB_B_Norm', 'DB_R_Norm', 'DB_A_Norm']

# 为每个密度值添加新字段用于存储标准化后的值
for new_field in new_fields:
    arcpy.AddField_management(feature_class, new_field, "FLOAT")

# 存储每种类型POI的密度值
densities = {field: [] for field in density_fields}

# 使用SearchCursor读取所有值
with arcpy.da.SearchCursor(feature_class, density_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            # 跳过None值
            if row[idx] is not None:
                densities[field].append(row[idx])

# 清理None值并计算最小值、最大值和范围
min_max_range = {}
for field, values in densities.items():
    cleaned_values = [x if x is not None else 0 for x in values]  # 清理None值
    min_val = np.min(cleaned_values)
    max_val = np.max(cleaned_values)
    range_val = max_val - min_val
    min_max_range[field] = (min_val, max_val, range_val)

# 使用UpdateCursor更新新字段的标准化值
with arcpy.da.UpdateCursor(feature_class, density_fields + new_fields) as cursor:
    for row in cursor:
        for idx, field in enumerate(density_fields):
            min_val, max_val, range_val = min_max_range[field]
            value = row[idx] if row[idx] is not None else 0  # 处理None值
            # 当范围为0时，所有值都相同，可以直接设置为0或1
            normalized_val = (value - min_val) / range_val if range_val else 0
            # 更新标准化值
            row[idx + len(density_fields)] = normalized_val
        cursor.updateRow(row)


In [20]:
# 如果用norm字段计算类型比例
norm_fields = ['DB_M_Norm', 'DB_G_Norm', 'DB_S_Norm', 'DB_B_Norm', 'DB_R_Norm', 'DB_A_Norm']
prop_fields = ['N2_M_Prop', 'N2_G_Prop', 'N2_S_Prop', 'N2_B_Prop', 'N2_R_Prop', 'N2_A_Prop']

# 为比例值添加新字段
for prop_field in prop_fields:
    arcpy.AddField_management(feature_class, prop_field, "DOUBLE")

# 计算并更新比例值
with arcpy.da.UpdateCursor(feature_class, norm_fields + prop_fields) as cursor:
    for row in cursor:
        # 计算所有标准化值的总和
        total_norm = sum(row[:len(norm_fields)])
        # 避免除以零
        if total_norm > 0:
            # 计算每个类型的比例值并更新行
            for i in range(len(norm_fields)):
                row[len(norm_fields) + i] = row[i] / total_norm
        else:
            # 如果总和为0，则将比例值设为平均值或其他适当的默认值
            for i in range(len(norm_fields), len(norm_fields) + len(prop_fields)):
                row[i] = 1.0 / len(norm_fields)  # 或其他适当的值
        cursor.updateRow(row)

In [21]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'N2'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

# 提取占比最高的两个字段

In [35]:
import arcpy

def find_top_two_categories(block_code, method_prefix, feature_class):
    """
    Identifies the top two categories of POI based on their proportions
    for a given block within the feature class.

    Args:
    - block_code: The unique identifier for the block.
    - method_prefix: The prefix used for differentiating methods in field names.
    - feature_class: The path to the feature class with the block data.
    """

    # Construct field names for POI density proportions
    poi_types = ['A', 'B', 'R', 'S', 'M', 'G']
    prop_field_names = [f"{method_prefix}_{pt}_Prop" for pt in poi_types]

    # 包括街区编码字段
    fields = ['Block_ID'] + prop_field_names
    print(fields)

    # 搜索特征类以找到特定街区并提取其比例值
    prop_values = []
    with arcpy.da.SearchCursor(feature_class, fields) as cursor:
        for row in cursor:
            if row[0] == block_code:  # 确保您使用的是字符串类型的编码
                # 提取该街区的比例值
                prop_values = [(field, row[i+1]) for i, field in enumerate(prop_field_names)]
                break
    print(prop_values)

    # 如果找到比例值
    if prop_values:
        # 根据值对比例进行降序排序
        prop_values.sort(key=lambda x: x[1], reverse=True)

        # 获取前两个类别及其值
        top_two = prop_values[:2]

        # 打印结果
        for rank, (category, value) in enumerate(top_two, start=1):
            # 从字段名称中提取类别类型
            category_type = category.split('_')[1]  # 提取类别类型（A、B、R、S、M、G）
            print(f"Rank {rank}: {category_type} with proportion value {value}")
    else:
        print(f"No data found for block code {block_code}.")

In [36]:
# Example usage
block_code = 29  # Replace with the actual block code
method_prefix = 'N'  # Replace with the actual method prefix
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'  # Replace with the actual feature class path

# Call the function
find_top_two_categories(block_code, method_prefix, feature_class)

['Block_ID', 'N_A_Prop', 'N_B_Prop', 'N_R_Prop', 'N_S_Prop', 'N_M_Prop', 'N_G_Prop']
[('N_A_Prop', 0.15007993259331426), ('N_B_Prop', 0.2737876923009916), ('N_R_Prop', 0.24254284832446435), ('N_S_Prop', 0.0), ('N_M_Prop', 0.3335895267812298), ('N_G_Prop', 0.0)]
Rank 1: M with proportion value 0.3335895267812298
Rank 2: B with proportion value 0.2737876923009916


In [ ]:
find_top_two_categories(255, 'N', feature_class)

In [24]:
import arcpy

# Set the path to the feature class
#feature_class = 'path_to_your_feature_class'

# Define the block code you are searching for
block_code_to_find = '29'

# Check if the block code exists in the feature class
block_code_exists = False
with arcpy.da.SearchCursor(feature_class, ['Block_ID']) as cursor:
    for row in cursor:
        if str(row[0]) == str(block_code_to_find):
            block_code_exists = True
            break

# Print the result
if block_code_exists:
    print(f"Block code {block_code_to_find} exists in the feature class.")
else:
    print(f"No data found for block code {block_code_to_find}.")


Block code 29 exists in the feature class.


In [39]:
import arcpy

def find_top_categories_for_methods(block_code, method_prefixes, feature_class, poi_types):
    """
    Finds the top two POI categories for a given block across multiple methods.
    
    Args:
    - block_code: The unique identifier for the block.
    - method_prefixes: A list of prefixes for different calculation methods.
    - feature_class: The path to the feature class with the block data.
    - poi_types: A list of POI types to consider.
    """

    # Loop through each method prefix
    for method_prefix in method_prefixes:
        # Construct field names for POI density proportions based on current method prefix
        prop_field_names = [f"{method_prefix}_{pt}_Prop" for pt in poi_types]

        # Include the block identifier field
        fields = ['Block_ID'] + prop_field_names

        # Initialize an empty list to store proportion values and their corresponding types
        prop_values = []

        # Search the feature class for the block and extract the proportion values
        with arcpy.da.SearchCursor(feature_class, fields) as cursor:
            for row in cursor:
                if row[0] == block_code:  # Ensure the block code matches
                    # Extract the proportion values for this block under current method
                    prop_values = [(field, row[i+1]) for i, field in enumerate(prop_field_names)]
                    break  # Exit loop after finding the block

        # Check if the proportions were found
        if not prop_values:
            print(f"No data found for block code {block_code} using method {method_prefix}.")
            continue  # Move on to the next method prefix

        # Sort the proportions in descending order
        prop_values.sort(key=lambda x: x[1], reverse=True)

        # Get the top two categories and their values
        top_two = prop_values[:2]

        # Print the results for current method
        print(f"Results for method {method_prefix}:")
        for rank, (category, value) in enumerate(top_two, start=1):
            category_type = category.split('_')[1]  # Extract the category type from the field name
            print(f"  Rank {rank}: {category_type} with proportion value {value}.")

In [37]:
method_prefixes = ['N', 'N0', 'N1', 'N2']  # Replace with the actual method prefixes
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'  # Replace with the actual feature class path
poi_types = ['A', 'B', 'R', 'S', 'M', 'G']  # The POI types to consider

In [68]:
find_top_categories_for_methods(29, method_prefixes, feature_class, poi_types)

Results for method N:
  Rank 1: M with proportion value 0.3335895267812298.
  Rank 2: B with proportion value 0.2737876923009916.
Results for method N0:
  Rank 1: B with proportion value 0.3187239511466901.
  Rank 2: M with proportion value 0.2988258099612053.
Results for method N1:
  Rank 1: M with proportion value 0.4313014963303365.
  Rank 2: B with proportion value 0.2429446185995513.
Results for method N2:
  Rank 1: M with proportion value 0.4454872308082931.
  Rank 2: B with proportion value 0.26358300708217.
